Added laser tables to `0_raw_laser` and `1_filtered_data` manually + excel formatting

In [64]:
import pandas as pd
import numpy as np
from ODR import odr_fit

### Length and height measurements

1. Read data

In [65]:
df = pd.read_excel("data.xlsx", sheet_name="1_filtered_data", index_col=0)
# Dropping measurements with very high errors
df = df.drop(list(range(20,31)) + [16, 18, 15, 0])
df

,picture_name,bottom_pixel,bottom_pixel_error,top_pixel,top_pixel_error,DC-volts,DC_error-volts,length-pixels,length_error-pixels,height-pixels,height_error-pixels
1,DSC03176.JPG,1635,15,1518,15,0.71537,0.0001,117,21.213203,1576.5,21.213203
2,DSC03177.JPG,1631,15,1466,15,0.75569,0.0001,165,21.213203,1548.5,21.213203
3,DSC03178.JPG,1628,15,1404,15,0.80554,0.0001,224,21.213203,1516.0,21.213203
4,DSC03179.JPG,1622,15,1346,15,0.85004,0.0001,276,21.213203,1484.0,21.213203
5,DSC03180.JPG,1617,15,1285,15,0.90011,0.0001,332,21.213203,1451.0,21.213203
6,DSC03181.JPG,1611,15,1221,15,0.95268,0.0001,390,21.213203,1416.0,21.213203
7,DSC03182.JPG,1610,15,1144,15,1.00995,0.0001,466,21.213203,1377.0,21.213203
8,DSC03183.JPG,1604,15,1089,15,1.05120,0.0001,515,21.213203,1346.5,21.213203
9,DSC03184.JPG,1597,15,1017,15,1.10904,0.0001,580,21.213203,1307.0,21.213203
10,DSC03185.JPG,1595,15,957,15,1.15601,0.0001,638,21.213203,1276.0,21.213203


In [66]:
results, chi_square, deegres_freedom, chi_square_reduced, p_value = odr_fit.perform_odr(
    df["DC-volts"].values,
    df["DC_error-volts"].values,
    df["length-pixels"].values,
    df["length_error-pixels"].values,
)

print(
    f"results {results},\nchi_square {chi_square},\ndeegres_freedom {deegres_freedom},\nchi_square_reduced {chi_square_reduced},\np_value {p_value}"
)

results <scipy.odr._odrpack.Output object at 0x11f0759d0>,
chi_square 0.2500845267329391,
deegres_freedom 12,
chi_square_reduced 0.020840377227744927,
p_value 0.9999999952300537


Plot length

In [67]:

odr_fit.plot_fit(
    df["DC-volts"].values,
    df["DC_error-volts"].values,
    df["length-pixels"].values,
    df["length_error-pixels"].values,
    results,
    "plot_fit.png",
)

Plot height

In [68]:
odr_fit.plot_residuals(
    df["DC-volts"].values,
    df["DC_error-volts"].values,
    df["length-pixels"].values,
    df["length_error-pixels"].values,
    results,
    "plot_residuals.png",
)

### Stick measurements
Get pixel length in mm

In [ ]:
df = pd.read_excel("data.xlsx", sheet_name="2_stick_measurements", index_col=0)

stick_diameter_in_mm = 125
stick_diameter_measurements = df["diameter_length-pixels"].values

# Calculate stick diameter in pixels, using mean
stick_diameter_in_pixels = np.mean(stick_diameter_measurements)

# Calculate the error:
# Calculating the standard error of the mean, applying Bessel's correction
std_dev = np.std(stick_diameter_measurements, ddof=1)
sem = std_dev / np.sqrt(len(stick_diameter_measurements))
# Getting the measurement error from the data - it is constant for all measurements
measurement_error_in_pixels = df["diameter_length_error-pixels"].values[0]
# Calculating the total error
stick_diameter_error_in_pixels = np.sqrt(measurement_error_in_pixels ** 2 + sem ** 2)

# Calculate the pixel to mm conversion factor
pixel_to_mm = stick_diameter_in_mm / stick_diameter_in_pixels
pixel_to_mm_error = pixel_to_mm * stick_diameter_error_in_pixels / stick_diameter_in_pixels

pixel_to_mm, pixel_to_mm_error

(np.float64(0.05934615365156852), np.float64(0.0001684304848405296))

### Particle mass measurement
Measure the particle's diameter

In [84]:
df = pd.read_excel("data.xlsx", sheet_name="3_particle_measurements", index_col=0)
particle_diameter_measurements = df["diameter-pixels"].values

# Calculate particle diameter in pixels
particle_diameter_in_pixels = np.mean(particle_diameter_measurements)

# Calculate the statistical error
particle_diameter_error_in_pixels = np.std(particle_diameter_measurements, ddof=1) / np.sqrt(len(particle_diameter_measurements))

# Particle diameter in mm
particle_diameter_in_mm = particle_diameter_in_pixels * pixel_to_mm
particle_diameter_error_in_mm = np.sqrt(
    (particle_diameter_error_in_pixels * pixel_to_mm) ** 2 + (particle_diameter_in_pixels * pixel_to_mm_error) ** 2
)

particle_diameter_in_mm, particle_diameter_error_in_mm

(np.float64(2.9475256313612364), np.float64(0.07181409158178655))

Calculate the particle's volume

In [ ]:
# Assuming the particle is a sphere:
particle_volume_in_mm_cubed = (np.pi * (particle_diameter_in_mm ** 3)) / 6
particle_volume_error_in_mm_cubed = particle_volume_in_mm_cubed * 3 * (particle_diameter_error_in_mm / particle_diameter_in_mm)

particle_volume_in_mm_cubed, particle_volume_error_in_mm_cubed

(np.float64(13.408228218918053), np.float64(0.9800420926126291))

Calculate the particle's mass using known density

In [91]:
PARTICLE_DENSITY_IN_KG_TO_M_CUBED = 510
PARTICLE_DENSITY_ERROR_IN_KG_TO_M_CUBED = 40

particle_volume_in_m_cubed = particle_volume_in_mm_cubed * 1e-9
particle_volume_error_in_m_cubed = particle_volume_error_in_mm_cubed * 1e-9

particle_mass_in_kg = particle_volume_in_m_cubed * PARTICLE_DENSITY_IN_KG_TO_M_CUBED
particle_mass_error_in_kg = np.sqrt(
    (particle_volume_error_in_m_cubed * PARTICLE_DENSITY_IN_KG_TO_M_CUBED) ** 2 + (particle_volume_in_m_cubed * PARTICLE_DENSITY_ERROR_IN_KG_TO_M_CUBED) ** 2
)

particle_mass_in_kg, particle_mass_error_in_kg

(np.float64(6.838196391648207e-06), np.float64(7.33123750440084e-07))

### Particle charge
